In [1]:
import contract_utilities.expiration as exp
report_date = exp.doubledate_shift_bus_days(shift_in_days=3)
date_list = exp.get_bus_day_list(date_from=20160915, date_to=report_date)
date_list

[20160915,
 20160916,
 20160919,
 20160920,
 20160921,
 20160922,
 20160923,
 20160926,
 20160927,
 20160928,
 20160929,
 20160930,
 20161003,
 20161004,
 20161005,
 20161006,
 20161007,
 20161010,
 20161011,
 20161012,
 20161013,
 20161014,
 20161017,
 20161018,
 20161019,
 20161020,
 20161021,
 20161024,
 20161025,
 20161026,
 20161027,
 20161028,
 20161031,
 20161101,
 20161102,
 20161103,
 20161104,
 20161107,
 20161108,
 20161109,
 20161110,
 20161111,
 20161114,
 20161115,
 20161116,
 20161117,
 20161118,
 20161121,
 20161122,
 20161123,
 20161125,
 20161128,
 20161129,
 20161130,
 20161201,
 20161202,
 20161205,
 20161206,
 20161207]

In [2]:
import opportunity_constructs.intraday_calendar_spreads as oics


for i in range(len(date_list)):
    print(date_list[i])
    oics.generate_ics_sheet_4date(date_to=date_list[i])

20160915
20160916
20160919
20160920
20160921
20160922
20160923
20160926
20160927
20160928
20160929
20160930
20161003
20161004
20161005
20161006
20161007
20161010
20161011
20161012
20161013
20161014
20161017
20161018
20161019
20161020
20161021
20161024
20161025
20161026
20161027
20161028
20161031
20161101
20161102
20161103
20161104
20161107
20161108
20161109
20161110
20161111
20161114
20161115
20161116
20161117
20161118
20161121
20161122
20161123
20161125
20161128
20161129
20161130
20161201
20161202
20161205
20161206
20161207


In [2]:

import backtesting.backtest_ics as bics
backtest_output_list = []
for i in range(len(date_list)):
        print(date_list[i])
        backtest_output_list.append(bics.backtest_continuous_ics_4date(report_date=date_list[i]))

20160915
20160916
20160919
20160920
20160921
20160922
20160923
20160926
20160927
20160928
20160929
20160930
20161003
20161004
20161005
20161006
20161007
20161010
20161011
20161012
20161013
20161014
20161017
20161018
20161019
20161020
20161021
20161024
20161025
20161026
20161027
20161028
20161031
20161101
20161102
20161103
20161104
20161107
20161108
20161109
20161110
20161111
20161114
20161115
20161116
20161117
20161118
20161121
20161122
20161123
20161125
20161128
20161129
20161130
20161201
20161202
20161205
20161206
20161207


In [17]:
import shared.statistics as stats
import pandas as pd
import numpy as np
big_data = pd.concat(backtest_output_list)
trade_data = big_data[big_data['ticker_head']=='NG']
#trade_data = big_data

stats.get_regression_results({'y':trade_data['pnl_long_morning'].values.astype(np.float64),
                              'x':trade_data['z5'].values.astype(np.float64)})

{'alpha': -51.626901302263775,
 'beta': 184.34257094427633,
 'conf_int': array([[ -62.6923408 ,  -40.56146181],
        [ 179.30750494,  189.37763695]]),
 'residualstd': 3153.9376505539799,
 'rsquared': 1.6224765979475975,
 'zscore': 1.1219572289810547}

In [9]:
trade_data['z1'].values.astype(np.float64)

array([-1.03365302, -1.03365302, -1.03365302, ...,  6.64694627,
        6.64694627,  6.64694627])

In [14]:
import pandas as pd
big_data = pd.concat(backtest_output_list)
selected_data = big_data[(big_data['z5']>2.6)&(big_data['hour_minute']==1000)&(big_data['ticker_head']=='FC')]
#selected_data = big_data[(big_data['z5']>2.6)&(big_data['ticker_head']=='FC')]

import matplotlib.pyplot as plt
plt.plot(selected_data['pnl_short_morning'].cumsum())
plt.grid()   
         
plt.show()

In [19]:
unique_dates = big_data['report_date'].unique()


short_pnl_list = []
long_pnl_list = []

for i in range(len(unique_dates)):
    daily_data = big_data[(big_data['report_date']==unique_dates[i])]
    sorted_data = daily_data.sort('z5')
    
    long_data = sorted_data[sorted_data['z5']>2.5]
    short_data = sorted_data[sorted_data['z5']<-2.5]
    short_pnl_list.append(short_data['pnl_short_afternoon_wc'].mean())
    long_pnl_list.append(long_data['pnl_long_afternoon_wc'].mean())


portfolio_frame = pd.DataFrame.from_items([('short_pnl', short_pnl_list), ('long_pnl', long_pnl_list)])
portfolio_frame['pnl'] = portfolio_frame['short_pnl']+portfolio_frame['long_pnl']

In [27]:
selected_data = big_data[(big_data['z5']<-1)]


import matplotlib.pyplot as plt
plt.plot(selected_data['pnl_long_afternoon'].cumsum())
plt.grid()   
         
plt.show()

In [20]:
import matplotlib.pyplot as plt
plt.plot(range(len(portfolio_frame.index)),portfolio_frame['pnl'].cumsum(),
         range(len(portfolio_frame.index)),portfolio_frame['short_pnl'].cumsum(),
         range(len(portfolio_frame.index)),portfolio_frame['long_pnl'].cumsum())
plt.grid()   
plt.legend(['total','short','long'])
         
plt.show()

In [17]:
16*portfolio_frame['short_pnl'].mean()/portfolio_frame['short_pnl'].std()

-2.8621141740178793

In [9]:
import pandas as pd
import backtesting.backtest_stats as bs
big_data = pd.concat(backtest_output_list)
#big_data['tickerHead'] = big_data['spread_description']

trade_data = big_data[big_data['ticker_head']=='FC']
#trade_data = big_data
bs.get_indicator_rr_table(trade_data=trade_data,indicator_name='z5',
                                strategy_class='ics',
                               long_pnl_field='pnl_long_afternoon_wc',
                               short_pnl_field='pnl_short_afternoon_wc')

,indicator_ulimit,mean_pnl,reward_risk
0,-1.698112,525.631969,0.199967
1,-0.866953,473.263370,0.214363
2,-0.189306,-1351.280055,-0.299058
3,0.486955,585.656960,0.381103
4,1.089164,842.841394,0.770612
5,1.410782,-442.827148,-0.244788
6,1.818593,-1268.024012,-0.564972
7,2.629166,-1135.048659,-0.480482
8,NaN,479.101634,0.207157


In [12]:
import numpy as np
np.isfinite(trade_data['pnl_long_morning'].values.astype(np.float64))

array([ True,  True,  True, ...,  True,  True,  True], dtype=bool)

In [18]:
import pandas as pd
import backtesting.backtest_stats as bs
big_data = pd.concat(backtest_output_list)
#big_data['tickerHead'] = big_data['spread_description']

#trade_data = big_data[big_data['ticker_head']=='NG']
trade_data = big_data
bs.get_indicator_rr_double_table(trade_data=trade_data,indicator_list=['z5','z1'],
                                strategy_class='ics',
                               long_pnl_field='pnl_long_morning',
                               short_pnl_field='pnl_short_morning')

,indicator1_ulimit,indicator2_ulimit,mean_pnl,reward_risk
0,NaN,NaN,74.148684,0.053684
1,NaN,3.401125,71.251517,0.056886
2,NaN,1.357766,51.517562,0.036062
3,0.802017,NaN,358.693598,0.379361
4,0.802017,0.712555,-69.272918,-0.083068
5,0.802017,-1.148152,-93.150351,-0.071448
6,-1.031872,NaN,-1.266422,-0.001184
7,-1.031872,-1.708064,272.770885,0.346838
8,-1.031872,-3.517816,386.973953,0.899714


In [12]:
big_data['pnl_short_morning'].cumsum()

2016-09-16 09:00:00       -238.279132
2016-09-16 09:01:00       -476.558263
2016-09-16 09:02:00       -714.837395
2016-09-16 09:03:00       -953.116527
2016-09-16 09:04:00      -1191.395658
2016-09-16 09:05:00      -1429.674790
2016-09-16 09:06:00      -1667.953922
2016-09-16 09:07:00      -1906.233053
2016-09-16 09:08:00      -2144.512185
2016-09-16 09:09:00      -2382.791317
2016-09-16 09:10:00      -2621.070448
2016-09-16 09:11:00      -2859.349580
2016-09-16 09:12:00      -3097.628712
2016-09-16 09:13:00      -3335.907843
2016-09-16 09:14:00      -3574.186975
2016-09-16 09:15:00      -3812.466107
2016-09-16 09:16:00      -4050.745239
2016-09-16 09:17:00      -4289.024370
2016-09-16 09:18:00      -4527.303502
2016-09-16 09:19:00      -4765.582634
2016-09-16 09:20:00      -5003.861765
2016-09-16 09:21:00      -5242.140897
2016-09-16 09:22:00      -5480.420029
2016-09-16 09:23:00      -5718.699160
2016-09-16 09:24:00      -5956.978292
2016-09-16 09:25:00      -6195.257424
2016-09-16 0

In [17]:
import backtesting.backtest_stats as bs
import pandas as pd
from IPython.display import display
big_data['tickerHead'] = big_data['ticker_head']
rank_indicators_output = bs.rank_indicators(trade_data=big_data,
                                            indicator_list = ['z1', 'z2', 'z5', 'z6'],
                                            long_pnl_field='pnl_long_morning',
                                            short_pnl_field='pnl_short_morning',
                                            strategy_class='ics',
                                            granular_ranking_type = 'ticker_head')
indicator_ranking_total = rank_indicators_output['indicator_ranking_total']
indicator_ranking_granular_total = rank_indicators_output['indicator_ranking_granular_total']
display(indicator_ranking_total)
display(indicator_ranking_granular_total)

,indicator,ranking
10,"[z5, z1]",22
11,"[z5, z2]",22
1,z2,18
4,"[z1, z2]",18
14,"[z6, z2]",18
7,"[z2, z1]",17
3,z6,15
0,z1,14
5,"[z1, z5]",14
6,"[z1, z6]",14


,indicator,ranking
14,"[z6, z2]",371
13,"[z6, z1]",365
11,"[z5, z2]",339
4,"[z1, z2]",330
0,z1,315
1,z2,314
15,"[z6, z5]",313
7,"[z2, z1]",308
10,"[z5, z1]",292
9,"[z2, z6]",288


In [15]:
import matplotlib.pyplot as plt
plt.plot(big_data['pnl_short_morning_wc'].cumsum())
plt.grid()         
         
plt.show()

In [2]:
import backtesting.backtest_ics as ics
backtest_results = []

for i in range(len(date_list)):
    print(date_list[i])
    backtest_results.append(ics.backtest_ics_4date(report_date=date_list[i]))


20160915
20160916
20160919
20160920
20160921
20160922
20160923
20160926
20160927
20160928
20160929
20160930
20161003
20161004
20161005
20161006
20161007
20161010
20161011
20161012
20161013
20161014
20161017
20161018
20161019
20161020
20161021
20161024
20161025
20161026
20161027
20161028
20161031
20161101
20161102
20161103
20161104
20161107
20161108
20161109
20161110
20161111
20161114
20161115
20161116
20161117
20161118
20161121
20161122


In [23]:
import pandas as pd
big_data = pd.concat(backtest_results)

In [31]:
big_data

,InstrumentName,MarketKey,ProductType,ProductName,Volume,ticker,ticker_head,downside,upside,ma_spread_low,ma_spread_high,intraday_sharp,pnl,num_trades,mean_holding_period
0,ICE_IPE IPE e-Brent Spread Nov16/Dec16,ICE_IPE,SPREAD,IPE e-Brent,48774,BX2016-BZ2016,B,-263.20000,254.10000,-7.450000e-03,4.750000e-03,-0.135335,3.600000e+01,6,3.000000
1,CME Calendar: 1xZL Oct16:-1xDec16,CME,SPREAD,ZL,6585,BOV2016-BOZ2016,BO,-37.47000,25.47000,-1.304512e-15,1.500000e-03,-0.138806,-2.000000e+00,2,7.500000
2,CME Calendar: 1xZC Dec16:-1xMar17,CME,SPREAD,ZC,23355,CZ2016-CH2017,C,-75.00000,62.50000,-3.437500e-02,2.812500e-02,0.111416,5.000000e+00,5,43.200000
3,ICE_IPE Cocoa Spread Dec16/Mar17,ICE_IPE,SPREAD,Cocoa,2170,CCZ2016-CCH2017,CC,-110.10000,140.15000,-4.625000e-01,1.170000e+00,-0.119577,-4.000000e+00,1,44.000000
4,CME Calendar: 1xCL Oct16:-1xNov16,CME,SPREAD,CL,72104,CLV2016-CLX2016,CL,-230.00000,268.90000,-7.000000e-03,7.750000e-03,-0.144160,-4.200000e+01,3,72.000000
5,ICE_IPE Cotton No 2 Spread Dec16/Mar17,ICE_IPE,SPREAD,Cotton No 2,995,CTZ2016-CTH2017,CT,-382.90000,293.42500,-2.202500e-02,2.620000e-02,-0.144339,5.500000e+00,3,24.000000
6,CME Calendar: 1xGE Sep16:-1xDec16,CME,SPREAD,GE,25091,EDU2016-EDZ2016,ED,-47.27500,33.72500,-1.687500e-03,1.750000e-03,-0.049245,-1.200000e+01,3,41.333333
7,CME Calendar: 1xGF Oct16:-1xNov16,CME,SPREAD,GF,900,FCV2016-FCX2016,FC,-400.18750,395.12500,-6.353125e-02,3.659375e-02,0.088213,4.875000e+01,5,25.800000
8,CME Calendar: 1xHO Oct16:-1xNov16,CME,SPREAD,HO,19637,HOV2016-HOX2016,HO,-88.32600,113.52600,-2.765000e-04,3.530000e-04,0.011121,-5.340000e+01,6,38.666667
9,ICE_IPE Coffee C Spread Dec16/Mar17,ICE_IPE,SPREAD,Coffee C,1069,KCZ2016-KCH2017,KC,-131.34375,159.46875,-2.220446e-15,2.975398e-14,NaN,NaN,NaN,NaN


In [40]:


selected_data = big_data[big_data['ticker_head']=='FC']
import matplotlib.pyplot as plt
plt.plot(selected_data['intraday_sharp'].cumsum())
plt.grid()         
         
plt.show()

In [38]:

selected_data = big_data[big_data['ticker_head']=='FC']
import matplotlib.pyplot as plt
plt.plot(selected_data['pnl'].cumsum())
plt.grid()         
         
plt.show()

In [ ]:
selected_data = big_data[big_data['ticker_head']=='FC']
import matplotlib.pyplot as plt
plt.plot(selected_data['pnl'].cumsum())
plt.grid()         
         
plt.show()

In [37]:
selected_data = big_data[big_data['ticker_head']=='B']
import matplotlib.pyplot as plt
plt.plot(selected_data['ma_spread_low'])
plt.grid()         
         
plt.show()

In [51]:

selected_data = big_data[big_data['ticker_head']=='C']
selected_data = big_data
import matplotlib.pyplot as plt
plt.plot(selected_data['intraday_sharp'].cumsum())
plt.grid()         
         
plt.show()